In [55]:
import nltk
import re
import math
from bs4 import BeautifulSoup
from pickle import dump, load
from nltk.corpus import cess_esp
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import stopwords

##############################################################
#					NORMALIZE TEXT
##############################################################

#Parameters: File path, encoding
#Return: String with only lower case letters
#Notes: path = '/Users/27AGO2019/Desktop/AbiiSnn/GitHub/Natural-Language-Processing/corpus/e961024.htm'
def getText(corpusRoot, code):
	f = open(corpusRoot, encoding = code) #Cod: utf-8, latin-1
	text = f.read()
	f.close()
	soup = BeautifulSoup(text, 'lxml')
	text = soup.get_text()
	text = text.lower()
	return text

#Parameters: Text
#Return: List of original tokens
def getTokens(text):
	tokens = nltk.word_tokenize(text)
	return tokens

#Parameters: List of normalize tokens
#Return: Set, vocabulary
def getVocabulary(tokens):
	s = set()
	for l in tokens:
		for element in l:
			s.add(element)
	vocabulary = sorted(s)
	return vocabulary

#Parameters: List of tuples of tokens
#Return: List of clean tokens and Tags
def getCleanTokensTags(tokens):
	clean = []
	for token in tokens:
		t = []
		for char in token[0]:
			if re.match(r'[a-záéíóúñüA-ZÁÉÍÓÚÜÑ]', char):
				t.append(char)
		letterToken = ''.join(t)

		if len(token[1]) > 0:
			tag = token[1]
			tag = tag[0].lower()

		if letterToken != '':
			l = (letterToken, tag)
			clean.append(l)
	return clean

#Parameters: List of clean tokens, language of stopwords
#Return: List of tokens without stopwords
def removeStopwords(tokens, language):
	sw = stopwords.words(language)

	cleanTokens = []
	for tok in tokens:
		l = ()
		if tok[0] not in sw:
			l = (tok[0], tok[1])
			cleanTokens.append(l)
	return cleanTokens

##############################################################
#						TAGGING
##############################################################
def make_and_save_combined_tagger(fname):
    default_tagger = nltk.DefaultTagger('v')
    patterns = [ (r'.*o$', 'n'),   # noun masculine singular
               	 (r'.*os$', 'n'),  # noun masculine plural
                 (r'.*a$', 'n'),   # noun feminine singular
                 (r'.*as$', 'n')   # noun feminine singular
               ]
    regexp_tagger = nltk.RegexpTagger(patterns, backoff=default_tagger)
    cess_tagged_sents = cess_esp.tagged_sents()
    combined_tagger = nltk.UnigramTagger(cess_tagged_sents, backoff=regexp_tagger)
    
    output = open(fname, 'wb')
    dump(combined_tagger, output, -1)
    output.close()

def tag(fname, text):
    input = open(fname, 'rb')
    default_tagger = load(input)
    input.close()

    s_tagged = default_tagger.tag(text)
    return s_tagged

##############################################################
#				     		LEMMAS
##############################################################
def getWord(word):
	cleanWord = ''
	for char in word:
		if char != '#':
			cleanWord += char
	return cleanWord

def getTag(word):
	c = 'v'
	if len(word) > 0:
		c = word[0]
	return c.lower()

# Return: Dictionary 
def createDicLemmas(tokensLemmas):
	lemmas = {}
	j = 0
	for i in range(0, len(tokensLemmas)- 2, 3):
		word = tokensLemmas[i]
		tag = tokensLemmas[i+1]
		val = tokensLemmas[i+2]
		l = (word, tag[0].lower())
		lemmas[l] = val
		j = j+1
	return lemmas

def lemmatizeText(tokens, lemmas):
	text = []
	for token in tokens:
		lemma = token[0]
		if token in lemmas:
			lemma = lemmas[token]
		aux = (lemma, token[1])
		text.append(aux)
	return text

##############################################################
#						CONTEXT
##############################################################

#Parameters: clean Tokens, vocabulary
#Return: Map of positions of every word in vocabulary
def initializeContext(tokens, vocabulary, lemmas):
	contexto = {}
	for word in vocabulary:
		contexto[word] = []

	for i in range(len(tokens)):
		contexto[tokens[i]].append(i)
	return contexto

#Parameters: Position of the word, size of window
#Return: Position of begin
def leftContextPosition(pos, window):
	pos = pos - window
	if(pos < 0):
		pos = 0
	return pos

#Parameters: Position of the word, size of window, size of window
#Return: Position of end
def rightContextPosition(pos, window, n):
	pos = pos + window
	if(pos > n):
		pos = n - 1 
	return pos

#Parameters: List of list(size 2)
#Return: list of context
def getContext(token, positions, window, originalText, vocabulary):
	context = []

	if token in positions:
		for pos in positions[token]:
			lpos = leftContextPosition(pos, window)
			rpos = rightContextPosition(pos, window, len(originalText))
			#con = []
			for i in range(lpos, pos):
				context.append(originalText[i])
			#con.append(token)
			for i in range(pos + 1, rpos):
				context.append(originalText[i])			
			#context.append(con)
	return context

##############################################################
#				VECTOR OPERATIONS
##############################################################

#Parameters: vector a, vector b
#Return: int, point product
#Notes: Need vectors of the same size
def pointProduct(a, b):
	ans = 0
	for i in range(0, len(a)):
		ans += (a[i] * b[i])
	return ans

#Parameters: vector of int
#Return: mag of a vector
def mag(v):
	ans = 0
	for i in range(0, len(v)):
		ans += (v[i] * v[i])
	return math.sqrt(ans)

##############################################################
#						FRECUENCY
##############################################################
def sumElements(vector):
	sum = 0
	for v in vector:
		sum = sum + v
	return sum

def getDocumentFrecuency(vocabulary, contexts):
	documentFreq = {}
	for term in vocabulary:
		documentFreq[term] = 0

	for context in contexts:
		for word in contexts[context]:
			documentFreq[word] = documentFreq[word] + 1 

	return documentFreq

def getIndexTuples(vocabulary):
	index = {}
	i = 0
	for term in vocabulary:
		index[term] = i 
		i = i + 1
	return index

def getFrecuency(vocabulary, contexts, b, k, documentFreq, lemmas, indexTuples):
	vectors = {}
	v = []
	for i in range(0, len(vocabulary) + 1):
		v.append(0)

	for term in vocabulary:
		vector = []
		for i in range(0, len(vocabulary) + 1):
			vector.append(0)
		for t in contexts[term]:
			vector[indexTuples[t]] = vector[indexTuples[t]] + 1
		vectors[term] = vector 

	sum = 0
	for element in vectors:
		for t in vectors[element]:
			sum = sum + t
	avdl = sum / len(vocabulary)
	
	# Getting IBM25 dictionary:
	IBM25 = {}
	for term in vocabulary:
		vectorFrecuency = vectors[term]
		magd1 = sumElements(vectorFrecuency)
		vIBM25 = []
		for t in vectorFrecuency:
			frec = 0
			num = (k + 1) * t 
			den = t + (k * (1 - b + ((b * magd1) / avdl)))
			if den != 0:
				frec = num / den
			vIBM25.append(frec)
		IBM25[term] = vIBM25

	# Getting IDF:
	IDF = []
	for term in vocabulary:
		frec = 0
		if documentFreq[term] != 0:
			frec = math.log((len(vocabulary) + 1) / documentFreq[term])
		IDF.append(frec)

	# Gettin tf - IDF = tf * IDF
	finalFrecuency = {}
	for term in vocabulary:
		vector = []
		ibm25 = IBM25[term]
		sumBM25 = sumElements(ibm25)
		for i in range(0, len(vocabulary)):
			frec = 0
			if sumBM25 != 0:
				frec = (ibm25[i] / sumBM25) * IDF[i]
			vector.append(frec)
		finalFrecuency[term] = vector

	return finalFrecuency

##############################################################
#						SIMILITUD
##############################################################
def getSimilitud(vocabulary, vectors, lemma):
	similitud = {}
	if lemma in vectors:
		v = vectors[lemma]
		for term in vocabulary:
			similitud[term] = 0
			if term in vectors:
				vec = vectors[term]
				if mag(v) != 0 and mag(vec) != 0:
					cos = pointProduct(v, vec) / (mag(v) * mag(vec))
					similitud[term] = cos
	return similitud

def getWords(fpath, code):
	f = open(fpath, encoding = code) #Cod: utf-8, latin-1
	text = f.read()
	f.close()

	words = re.sub(" ", " ",  text).split()
	# words = text.words(fname)
	# words = list(words) #Convertir a lista de palabras
	return words

def filtredSimilitud(similitud, word):
	sim = {}
	for t in similitud:
		if t[1] == word[1]:
			sim[t] = similitud[t]
	return sim

##############################################################
#						CREATE FILE
##############################################################

#Parameters: Set, vocabulary
#Return: Nothing
def createFile(path, vocabulary):
	f = open(path, 'w')
	for word in vocabulary:
		f.write(word + '\n')
	f.close()

#Parameters: , vocabulary
#Return: Nothing
def createFileDic(path, l):
	f = open(path, 'w')
	for item in l:
		f.write(str(item))
		f.write('\n')
	f.close()

def printContext(context):
	for i in range(0, len(context)):
		aux = ''
		for j in range(0, len(context[i])):
			aux += context[i][j] + " "
		print(aux)

def printDictionary(dic, n):
	i = 0
	for j in dic:
		print(j, dic[j])
		i = i + 1
		if i > n:
			break

def makePKL(fname, aux):
	output = open(fname, 'wb')
	dump(aux, output, -1)
	output.close()

def getPKL(fname):
	input = open(fname, 'rb')
	aux = load(input)
	input.close()
	return aux

In [56]:
def get_sentences(fname):
    f=open(fname, encoding = 'utf-8')
    t=f.read()
    soup = BeautifulSoup(t, 'lxml')
    text_string = soup.get_text()

    #get a list of sentences
    sent_tokenizer=nltk.data.load('nltk:tokenizers/punkt/english.pickle')
    sentences=sent_tokenizer.tokenize(text_string)
    return sentences

In [57]:
def getCount(sentences, vocabulary):
 	count = {}
 	for i in vocabulary:
 		count[i] = 0
 	for sentence in sentences:
 		for element in sentence:
 			if element in vocabulary:
	 			count[element] = count[element] + 1
 	return count

In [86]:
def getConjunta(sentences, w1, w2, p2):
	ans = 0
	for sentence in sentences:
		v = sentence
		w1count = v.count(w1)
		w2count = v.count(w2)
		if (w1count == 1) and (w2count == 1):
			ans = ans + 1
	print(str(ans) + "/" + str(p2))
	return ans / p2

In [59]:
def Hw1w2(w1, w2, p11):
	w10 = 1 - w1
	w20 = 1 - w2
	p01 = w2 - p11
	p10 = w1 - p11
	p00 = w10 - p01
	a = 0
	b = 0
	c = 0
	d = 0
	if w20 != 0:
		if (p00 / w20) > 0:
			a = p00 * math.log2(p00 / w20)
		if (p10 / w20) > 0:
			b = p10 * math.log2(p10 / w20)
	if w2 != 0:
		if (p01 / w2) > 0:
			c = p01 * math.log2(p01 / w2)
		if (p11 / w2) > 0:
			d = p11 * math.log2(p11 / w2)

	return -1 * (a + b + c + d)

In [60]:
fpathLemmas = './Normalize/generateClean.txt'
fpathName = 'generateClean.txt'
nameFile = './Normalize/similitudLemma.txt'
code = 'ISO-8859-1'

In [61]:
textLemmas = getWords(fpathLemmas, code)
print(textLemmas[:100])

['a', 'SPS00', 'a', 'a-já', 'I', 'a-já', 'abad', 'NCMS000', 'abad', 'abades', 'NCMP000', 'abad', 'abadesa', 'NCFS000', 'abadesa', 'abadesas', 'NCFP000', 'abadesa', 'abajo', 'I', 'abajo', 'abajo', 'RG000', 'abajo', 'abalancé', 'V0IS1S0', 'abalanzar', 'abalance', 'V0SP1S0', 'abalanzar', 'abalances', 'V0SP2S0', 'abalanzar', 'abalance', 'V0SP3S0', 'abalanzar', 'abalancemos', 'V0SP1P0', 'abalanzar', 'abalancéis', 'V0SP2P0', 'abalanzar', 'abalancen', 'V0SP3P0', 'abalanzar', 'abalance', 'V0R03S0', 'abalanzar', 'abalánceme', 'V0R03S0', 'abalanzar', 'abaláncese', 'V0R03S0', 'abalanzar', 'abaláncelo', 'V0R03S0', 'abalanzar', 'abaláncela', 'V0R03S0', 'abalanzar', 'abaláncelos', 'V0R03S0', 'abalanzar', 'abaláncelas', 'V0R03S0', 'abalanzar', 'abaláncele', 'V0R03S0', 'abalanzar', 'abalánceles', 'V0R03S0', 'abalanzar', 'abaláncenos', 'V0R03S0', 'abalanzar', 'abaláncemelo', 'V0R03S0', 'abalanzar', 'abaláncemela', 'V0R03S0', 'abalanzar', 'abaláncemelos', 'V0R03S0', 'abalanzar', 'abaláncemelas', 'V0R03S

In [62]:
lemmas = {}
lemmas = createDicLemmas(textLemmas)
print(list(lemmas.items())[:100])

[(('a', 's'), 'a'), (('a-já', 'i'), 'a-já'), (('abad', 'n'), 'abad'), (('abades', 'n'), 'abad'), (('abadesa', 'n'), 'abadesa'), (('abadesas', 'n'), 'abadesa'), (('abajo', 'i'), 'abajo'), (('abajo', 'r'), 'abajo'), (('abalancé', 'v'), 'abalanzar'), (('abalance', 'v'), 'abalanzar'), (('abalances', 'v'), 'abalanzar'), (('abalancemos', 'v'), 'abalanzar'), (('abalancéis', 'v'), 'abalanzar'), (('abalancen', 'v'), 'abalanzar'), (('abalánceme', 'v'), 'abalanzar'), (('abaláncese', 'v'), 'abalanzar'), (('abaláncelo', 'v'), 'abalanzar'), (('abaláncela', 'v'), 'abalanzar'), (('abaláncelos', 'v'), 'abalanzar'), (('abaláncelas', 'v'), 'abalanzar'), (('abaláncele', 'v'), 'abalanzar'), (('abalánceles', 'v'), 'abalanzar'), (('abaláncenos', 'v'), 'abalanzar'), (('abaláncemelo', 'v'), 'abalanzar'), (('abaláncemela', 'v'), 'abalanzar'), (('abaláncemelos', 'v'), 'abalanzar'), (('abaláncemelas', 'v'), 'abalanzar'), (('abaláncemele', 'v'), 'abalanzar'), (('abaláncemeles', 'v'), 'abalanzar'), (('abalánceselo'

In [63]:
fpath = './files/e961024.htm'
sentences = get_sentences(fpath)

In [64]:
print(sentences)

['\n\n\ne961024_mod.htm\n\n\n\nhttp://www.excelsior.com.mx/9610/961024/art01.html\n\n\nExcelsior: Editorial\n\n\nJueves 24 de Octubre de 1996\n\nEpigrama\n\nJORGE MANSILLA TORRES\n\nCritica el Miami Herald al Presidente ecuatoriano, por autoproclamarse loco.', 'El neoliberalismo\nque se aplica sin encomio\nhace que sean lo mismo\nun país y un manicomio.', 'Editorial -\nNota siguiente\n\nhttp://www.excelsior.com.mx/9610/961024/art02.html\n\n\nExcelsior: Editorial\n\n\nJueves 24 de Octubre de 1996\n\nHungría\n\nLa Rebelión Antiestalinista de 1956\n\nOSCAR GONZALEZ LOPEZ\n\nEN el curso de octubre de 1956, una insurgencia popular comandada por estudiantes, intelectuales y obreros partidarios de establecer en suelo húngaro un régimen socialista con libertad, creció vertiginosamente hasta desembocar en una auténtica insurrección armada contra la estructura estalinista del Estado surgido como consecuencia de los resultados de la Primera Guerra Mundial.', 'A 40 años de distancia, conviene anal

In [65]:
make_and_save_combined_tagger(fcombinedTagger)
sentencesTag = []
for i in range(0, len(sentences)):
	aux = getTokens(sentences[i])
	auxTag = tag(fcombinedTagger, aux)
	sentencesTag.append(auxTag)

In [66]:
print(sentencesTag[:200])

[[('e961024_mod.htm', 'v'), ('http', 'v'), (':', 'Fd'), ('//www.excelsior.com.mx/9610/961024/art01.html', 'v'), ('Excelsior', 'v'), (':', 'Fd'), ('Editorial', 'v'), ('Jueves', 'v'), ('24', 'Z'), ('de', 'sps00'), ('Octubre', 'v'), ('de', 'sps00'), ('1996', 'W'), ('Epigrama', 'n'), ('JORGE', 'v'), ('MANSILLA', 'v'), ('TORRES', 'v'), ('Critica', 'n'), ('el', 'da0ms0'), ('Miami', 'np0000l'), ('Herald', 'v'), ('al', 'spcms'), ('Presidente', 'np0000p'), ('ecuatoriano', 'aq0ms0'), (',', 'Fc'), ('por', 'sps00'), ('autoproclamarse', 'v'), ('loco', 'aq0ms0'), ('.', 'Fp')], [('El', 'da0ms0'), ('neoliberalismo', 'n'), ('que', 'pr0cn000'), ('se', 'p0300000'), ('aplica', 'vmip3s0'), ('sin', 'sps00'), ('encomio', 'n'), ('hace', 'vmip3s0'), ('que', 'pr0cn000'), ('sean', 'vssp3p0'), ('lo', 'da0ns0'), ('mismo', 'di0ms0'), ('un', 'di0ms0'), ('país', 'ncms000'), ('y', 'cc'), ('un', 'di0ms0'), ('manicomio', 'n'), ('.', 'Fp')], [('Editorial', 'v'), ('-', 'Fg'), ('Nota', 'n'), ('siguiente', 'aq0cs0'), ('http

In [67]:
sentencesCleanTokens = []
for i in range(0, len(sentencesTag)):
	aux = getCleanTokensTags(sentencesTag[i])
	sentencesCleanTokens.append(aux)

In [68]:
print(sentencesCleanTokens[:100])

[[('emodhtm', 'v'), ('http', 'v'), ('wwwexcelsiorcommxarthtml', 'v'), ('Excelsior', 'v'), ('Editorial', 'v'), ('Jueves', 'v'), ('de', 's'), ('Octubre', 'v'), ('de', 's'), ('Epigrama', 'n'), ('JORGE', 'v'), ('MANSILLA', 'v'), ('TORRES', 'v'), ('Critica', 'n'), ('el', 'd'), ('Miami', 'n'), ('Herald', 'v'), ('al', 's'), ('Presidente', 'n'), ('ecuatoriano', 'a'), ('por', 's'), ('autoproclamarse', 'v'), ('loco', 'a')], [('El', 'd'), ('neoliberalismo', 'n'), ('que', 'p'), ('se', 'p'), ('aplica', 'v'), ('sin', 's'), ('encomio', 'n'), ('hace', 'v'), ('que', 'p'), ('sean', 'v'), ('lo', 'd'), ('mismo', 'd'), ('un', 'd'), ('país', 'n'), ('y', 'c'), ('un', 'd'), ('manicomio', 'n')], [('Editorial', 'v'), ('Nota', 'n'), ('siguiente', 'a'), ('http', 'v'), ('wwwexcelsiorcommxarthtml', 'v'), ('Excelsior', 'v'), ('Editorial', 'v'), ('Jueves', 'v'), ('de', 's'), ('Octubre', 'v'), ('de', 's'), ('Hungría', 'n'), ('La', 'd'), ('Rebelión', 'v'), ('Antiestalinista', 'n'), ('de', 's'), ('OSCAR', 'v'), ('GONZAL

In [69]:
# Remove Stopwords
language = 'spanish'
sentencesClean = []
for i in range(0, len(sentencesCleanTokens)):
	aux = removeStopwords(sentencesCleanTokens[i], language)
	sentencesClean.append(aux)

In [70]:
print(sentencesClean[:50])

[[('emodhtm', 'v'), ('http', 'v'), ('wwwexcelsiorcommxarthtml', 'v'), ('Excelsior', 'v'), ('Editorial', 'v'), ('Jueves', 'v'), ('Octubre', 'v'), ('Epigrama', 'n'), ('JORGE', 'v'), ('MANSILLA', 'v'), ('TORRES', 'v'), ('Critica', 'n'), ('Miami', 'n'), ('Herald', 'v'), ('Presidente', 'n'), ('ecuatoriano', 'a'), ('autoproclamarse', 'v'), ('loco', 'a')], [('El', 'd'), ('neoliberalismo', 'n'), ('aplica', 'v'), ('encomio', 'n'), ('hace', 'v'), ('mismo', 'd'), ('país', 'n'), ('manicomio', 'n')], [('Editorial', 'v'), ('Nota', 'n'), ('siguiente', 'a'), ('http', 'v'), ('wwwexcelsiorcommxarthtml', 'v'), ('Excelsior', 'v'), ('Editorial', 'v'), ('Jueves', 'v'), ('Octubre', 'v'), ('Hungría', 'n'), ('La', 'd'), ('Rebelión', 'v'), ('Antiestalinista', 'n'), ('OSCAR', 'v'), ('GONZALEZ', 'v'), ('LOPEZ', 'v'), ('EN', 's'), ('curso', 'n'), ('octubre', 'w'), ('insurgencia', 'n'), ('popular', 'a'), ('comandada', 'n'), ('estudiantes', 'n'), ('intelectuales', 'n'), ('obreros', 'n'), ('partidarios', 'n'), ('esta

In [71]:
# Lemmatize text
tokens = []
for i in range(0, len(sentencesClean)):
	aux = lemmatizeText(sentencesClean[i], lemmas)
	tokens.append(aux)

In [72]:
print(tokens[:40])

[[('emodhtm', 'v'), ('http', 'v'), ('wwwexcelsiorcommxarthtml', 'v'), ('Excelsior', 'v'), ('Editorial', 'v'), ('Jueves', 'v'), ('Octubre', 'v'), ('Epigrama', 'n'), ('JORGE', 'v'), ('MANSILLA', 'v'), ('TORRES', 'v'), ('Critica', 'n'), ('Miami', 'n'), ('Herald', 'v'), ('Presidente', 'n'), ('ecuatoriano', 'a'), ('autoproclamarse', 'v'), ('loco', 'a')], [('El', 'd'), ('neoliberalismo', 'n'), ('aplicar', 'v'), ('encomio', 'n'), ('hacer', 'v'), ('mismo', 'd'), ('país', 'n'), ('manicomio', 'n')], [('Editorial', 'v'), ('Nota', 'n'), ('siguiente', 'a'), ('http', 'v'), ('wwwexcelsiorcommxarthtml', 'v'), ('Excelsior', 'v'), ('Editorial', 'v'), ('Jueves', 'v'), ('Octubre', 'v'), ('Hungría', 'n'), ('La', 'd'), ('Rebelión', 'v'), ('Antiestalinista', 'n'), ('OSCAR', 'v'), ('GONZALEZ', 'v'), ('LOPEZ', 'v'), ('EN', 's'), ('curso', 'n'), ('octubre', 'w'), ('insurgencia', 'n'), ('popular', 'a'), ('comandada', 'n'), ('estudiante', 'n'), ('intelectuales', 'n'), ('obrero', 'n'), ('partidario', 'n'), ('estab

In [73]:
tokensUnic = []
for i in range(0, len(tokens)):
	aux = sorted(set(tokens[i]))
	tokensUnic.append(aux)

In [74]:
print(tokensUnic[:30])

[[('Critica', 'n'), ('Editorial', 'v'), ('Epigrama', 'n'), ('Excelsior', 'v'), ('Herald', 'v'), ('JORGE', 'v'), ('Jueves', 'v'), ('MANSILLA', 'v'), ('Miami', 'n'), ('Octubre', 'v'), ('Presidente', 'n'), ('TORRES', 'v'), ('autoproclamarse', 'v'), ('ecuatoriano', 'a'), ('emodhtm', 'v'), ('http', 'v'), ('loco', 'a'), ('wwwexcelsiorcommxarthtml', 'v')], [('El', 'd'), ('aplicar', 'v'), ('encomio', 'n'), ('hacer', 'v'), ('manicomio', 'n'), ('mismo', 'd'), ('neoliberalismo', 'n'), ('país', 'n')], [('Antiestalinista', 'n'), ('EN', 's'), ('Editorial', 'v'), ('Estado', 'n'), ('Excelsior', 'v'), ('GONZALEZ', 'v'), ('Guerra', 'n'), ('Hungría', 'n'), ('Jueves', 'v'), ('LOPEZ', 'v'), ('La', 'd'), ('Mundial', 'n'), ('Nota', 'n'), ('OSCAR', 'v'), ('Octubre', 'v'), ('Primera', 'n'), ('Rebelión', 'v'), ('armado', 'a'), ('auténtico', 'a'), ('comandada', 'n'), ('consecuencia', 'n'), ('crecer', 'v'), ('curso', 'n'), ('desembocar', 'v'), ('establecer', 'v'), ('estalinista', 'n'), ('estructura', 'n'), ('estu

In [75]:
vocabulary = getVocabulary(tokensUnic)

In [76]:
print(vocabulary[:40])

[('A', 's'), ('A', 'v'), ('ACAPULCO', 'v'), ('ACP', 'n'), ('ADN', 'n'), ('ADORNO', 'v'), ('ADR', 'v'), ('AFP', 'v'), ('AGUSTIN', 'v'), ('AHORA', 'v'), ('AIG', 'v'), ('AJEDREZ', 'v'), ('AL', 's'), ('ALBERTO', 'v'), ('ALDF', 'v'), ('ALEXEI', 'v'), ('ALFREDO', 'v'), ('ALPONTE', 'v'), ('AN', 'v'), ('ANDRES', 'v'), ('ANTONIO', 'v'), ('AP', 'v'), ('APDJ', 'v'), ('APDow', 'v'), ('APL', 'v'), ('APOYOS', 'v'), ('ARE', 'v'), ('ARIAS', 'v'), ('ARVIZU', 'v'), ('ASOMBRA', 'v'), ('AT', 'v'), ('ATLANTA', 'v'), ('AURELIO', 'v'), ('AURORA', 'v'), ('AbascadoCarranza', 'n'), ('Abastecerá', 'v'), ('Abasto', 'n'), ('Abracadabrante', 'v'), ('Abrir', 'v'), ('Absolutamente', 'v')]


In [84]:
count = {}
count = getCount(tokensUnic, vocabulary)
printDictionary(count, 100)

('A', 's') 37
('A', 'v') 17
('ACAPULCO', 'v') 1
('ACP', 'n') 2
('ADN', 'n') 1
('ADORNO', 'v') 1
('ADR', 'v') 2
('AFP', 'v') 2
('AGUSTIN', 'v') 1
('AHORA', 'v') 1
('AIG', 'v') 1
('AJEDREZ', 'v') 1
('AL', 's') 1
('ALBERTO', 'v') 5
('ALDF', 'v') 2
('ALEXEI', 'v') 1
('ALFREDO', 'v') 1
('ALPONTE', 'v') 1
('AN', 'v') 2
('ANDRES', 'v') 1
('ANTONIO', 'v') 4
('AP', 'v') 1
('APDJ', 'v') 1
('APDow', 'v') 5
('APL', 'v') 2
('APOYOS', 'v') 1
('ARE', 'v') 2
('ARIAS', 'v') 1
('ARVIZU', 'v') 1
('ASOMBRA', 'v') 1
('AT', 'v') 1
('ATLANTA', 'v') 1
('AURELIO', 'v') 3
('AURORA', 'v') 1
('AbascadoCarranza', 'n') 1
('Abastecerá', 'v') 1
('Abasto', 'n') 1
('Abracadabrante', 'v') 1
('Abrir', 'v') 1
('Absolutamente', 'v') 1
('Abundó', 'v') 1
('Academia', 'n') 1
('Acarrearía', 'n') 1
('Acceso', 'n') 1
('Acciones', 'v') 1
('Acción', 'v') 1
('Aceite', 'v') 1
('Aceptan', 'v') 1
('Acetilmetoxitriptamina', 'n') 1
('Aceves', 'v') 1
('Acompañado', 'n') 1
('Activos', 'n') 1
('Actualmente', 'r') 3
('Acudir', 'v') 1
('Acue

In [79]:
w1 = ('economía', 'n')
p1 = count[w1] / len(sentences)
print(p1)

0.016445623342175066


In [87]:
ans = {}
p1 = count[w1] / len(sentences)
for w2 in vocabulary:
    p2 = count[w2] / len(sentences)
    p1p2 = getConjunta(tokensUnic, w1, w2, len(sentences))
    ans[w2] = Hw1w2(p1, p2, p1p2)

2/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885

0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
2/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
4/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
3/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885

0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
3/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
4/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885

0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
1/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885

0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
2/1885
0/1885
0/1885
0/1885
4/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
3/1885
0/1885
0/1885
0/1885
1/1885

0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
2/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885

0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
5/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
1/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885
0/1885

In [81]:
print(list(ans.items())[:20])

[(('A', 's'), 0.12018689236886909), (('A', 'v'), 0.1207723774239291), (('ACAPULCO', 'v'), 0.12097642334215743), (('ACP', 'n'), 0.12096372175524711), (('ADN', 'n'), 0.12097642334215743), (('ADORNO', 'v'), 0.12097642334215743), (('ADR', 'v'), 0.12096372175524711), (('AFP', 'v'), 0.12096372175524711), (('AGUSTIN', 'v'), 0.12097642334215743), (('AHORA', 'v'), 0.12097642334215743), (('AIG', 'v'), 0.12097642334215743), (('AJEDREZ', 'v'), 0.12097642334215743), (('AL', 's'), 0.12097642334215743), (('ALBERTO', 'v'), 0.12092557614421653), (('ALDF', 'v'), 0.12096372175524711), (('ALEXEI', 'v'), 0.12097642334215743), (('ALFREDO', 'v'), 0.12097642334215743), (('ALPONTE', 'v'), 0.12097642334215743), (('AN', 'v'), 0.12096372175524711), (('ANDRES', 'v'), 0.12097642334215743)]


In [82]:
l = list()
for key, val in ans.items():
	l.append((val, key))
l.sort(reverse = True)

[(0.12098910506477176, ('En', 's')), (0.12098909693819157, ('país', 'n')), (0.12098823508584056, ('Los', 'd')), (0.12098475040200428, ('millón', 'n')), (0.12098335464444669, ('hacer', 'v')), (0.12097979295046857, ('mayor', 'a')), (0.12097642334215743, ('últimamente', 'r')), (0.12097642334215743, ('órgano', 'n')), (0.12097642334215743, ('óptima', 'n')), (0.12097642334215743, ('índole', 'n'))]


In [83]:
for val,token in l[:100]:
    print(str(val) + ", " + str(token))

0.12098910506477176, ('En', 's')
0.12098909693819157, ('país', 'n')
0.12098823508584056, ('Los', 'd')
0.12098475040200428, ('millón', 'n')
0.12098335464444669, ('hacer', 'v')
0.12097979295046857, ('mayor', 'a')
0.12097642334215743, ('últimamente', 'r')
0.12097642334215743, ('órgano', 'n')
0.12097642334215743, ('óptima', 'n')
0.12097642334215743, ('índole', 'n')
0.12097642334215743, ('éticas', 'n')
0.12097642334215743, ('ética', 'n')
0.12097642334215743, ('ágil', 'a')
0.12097642334215743, ('á', 'v')
0.12097642334215743, ('zarandear', 'v')
0.12097642334215743, ('zapatista', 'a')
0.12097642334215743, ('zanja', 'n')
0.12097642334215743, ('yy', 'v')
0.12097642334215743, ('yugulamiento', 'n')
0.12097642334215743, ('yte', 'v')
0.12097642334215743, ('yacimiento', 'n')
0.12097642334215743, ('vínculo', 'n')
0.12097642334215743, ('víbora', 'n')
0.12097642334215743, ('válvula', 'n')
0.12097642334215743, ('vuelta', 'n')
0.12097642334215743, ('vuelo', 'n')
0.12097642334215743, ('votaría', 'n')
0.120